# MySQL Nested Queries

In [1]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/{database}?charset=utf8'.format(
    host = 'mysql-techub-2300010003-spring.db', 
    user = 'dbreader',
    password = 'ub232023',
    database = 'imdb')

engine = create_engine(conn_string)
con = engine.connect()

Prepare sql_magic library that enable to query to database easily.

In [2]:
%reload_ext sql_magic
%config SQL.conn_name = 'engine'

#### CAUTION! PLEASE RUN THIS CELL! This cell limits the maximum number of records to obtain.

In [10]:
%%read_sql
SET sql_safe_updates=1, sql_select_limit=1000, max_join_size=10000000;

Query started at 09:31:56 PM EDT; Query executed in 0.01 m

Now we are all set! Let us start querying data from IMDB database.





In [21]:
%%read_sql
USE imdb;

Query started at 10:38:13 AM EDT; Query executed in 0.01 m

## Session starts here.

See also
> https://www.imdb.com/interfaces/

#### Nested queries



JOIN queries can be **rewritten** by using subqueries. Revisit the query of S3: The following query return the average rating and numvotes of Sherlock (2010).

In [22]:
%%read_sql
SELECT averageRating, numVotes
FROM TitleRatings r
INNER JOIN TitleBasics b ON r.tconst = b.tconst
WHERE
originalTitle = "Sherlock"
AND
startYear = 2010
;

Query started at 10:38:19 AM EDT; Query executed in 0.02 m

,averageRating,numVotes
0,9.1,941117


The following query, which uses subquery, returns the same information.

In [23]:
%%read_sql
SELECT averageRating, numVotes
FROM TitleRatings
WHERE tconst =
(
  SELECT tconst FROM TitleBasics
  WHERE
  originalTitle = "Sherlock"
  AND
  startYear = 2010    
)
;

Query started at 10:38:23 AM EDT; Query executed in 0.00 m

,averageRating,numVotes
0,9.1,941117


This query above is **nested** because one query is inside another. The inner query, or **subquery** -- the one that is nested -- is written in **parentheses**, and you can see that it determines the tconst in a silent way. The outer query is the one that is listed first and is not parenthesized here.

#### Exercise

Similar to the exercise of S3, create queries that answers to the following questions Q1-Q3. Here, instead of using inner join, use **nested queries**.

```
Q1. Find a movie with its **primaryTitle** "Les Miserables"
      and **startYear**  2012 in **TitleBasics** table.
Q2. Find the **averageRating** of the movie by using the query of Q1 as a  
      subquery and write an outer query for **TitleRatings** table.
Q3. Find the **directors** of the movie by using the query of Q1 as a 
      subquery and write outer query for **TitleCrew** table.
    This should return "nmXXXXXXX". 
```


In [ ]:
# Your answer to Q1 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

In [ ]:
# Your answer to Q2 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

In [ ]:
# Your answer to Q3 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

#### Some queries are easier to do in nested query. 

The pros of nested queries: 

*   Nested queries are expressive: Readability is higher once you get used to it.
*   Nested queries are even more powerful in some sort of queries.

The cons of nested queries:

*   Usually, nested queries are slower than non-nested queries.


The next query is an example where nested query makes it much easier. Let us first check the largest numVotes.

In [14]:
%%read_sql
SELECT max(numVotes)
FROM TitleRatings;

Query started at 09:32:08 PM EDT; Query executed in 0.00 m

,max(numVotes)
0,2714675


More specifically, we want the **primaryTitle** that has largest numVotes. Can we do like below?

In [24]:
# NOTE: This cell doesn't work (ERROR)! The two attrs, max(numVotes) (aggregation column) and primaryTitle (standard column) cannot co-exists.
%%read_sql
SELECT max(numVotes), primaryTitle
FROM TitleRatings r
INNER JOIN TitleBasics b
ON r.tconst = b.tconst
;

SyntaxError: invalid syntax (2231511896.py, line 3)

We can fix it by using by JOIN query and directly specifying numVotes (given that we obtained it in another query).

In [27]:
%%read_sql
SELECT *
FROM TitleBasics b
INNER JOIN TitleRatings r
ON b.tconst = r.tconst
where numVotes = 2714675
;

Query started at 10:38:41 AM EDT; Query executed in 0.00 m

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tconst,averageRating,numVotes
0,108666,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,None,142,Drama,tt0111161,9.3,2714675


Moreover, we can do it in just one query, where we obtain max(numVotes) from another query:

In [26]:
%%read_sql
SELECT *
FROM TitleBasics b
INNER JOIN TitleRatings r
ON b.tconst = r.tconst
where numVotes = 
(
  SELECT max(numVotes)
   FROM
  TitleRatings
)
;

Query started at 10:38:34 AM EDT; Query executed in 0.00 m

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tconst,averageRating,numVotes
0,108666,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,None,142,Drama,tt0111161,9.3,2714675


#### Doubly-nested query

The following query is an example of doubly-nested query that gives us a list of movies "Hugh Jackman" is involving. We first (1) `nconst` of Hugh Jackman, then we (2) obtain the list of `tconst`s that Hugh Jackman is appearing, and finally (3) obtain the primaryTitle of the selected `tconst`s.

In [29]:
%%read_sql
SELECT primaryTitle FROM TitleBasics
WHERE tconst IN
(
    SELECT tconst FROM TitlePrincipals WHERE nconst IN
    (
        SELECT nconst FROM NameBasics WHERE
          primaryName = "Hugh Jackman"
    )
)
AND 
  titleType = "movie"
;

Query started at 10:41:32 AM EDT; Query executed in 0.01 m

,primaryTitle
0,Kate & Leopold
1,X-Men
2,Paperback Hero
3,Erskineville Kings
4,Swordfish
5,Someone Like You
6,X2: X-Men United
7,Van Helsing
8,Happy Feet
9,X-Men: The Last Stand


#### Variables

While subqueries enable us to pass the results of one query to another, we often want to save them so that we can use later. User variables provides us a way to do that. Let us start with the above example of largest numVotes.

The following query assigned the results of the query to a variable `@largestNumVotes`.

In [ ]:
%%read_sql
SET @largestNumVotes = 
(
    SELECT MAX(numVotes) FROM TitleRatings
);

We can check the variable like:

In [ ]:
%%read_sql
SELECT @largestNumVotes;

And reuse it in the following query:

In [ ]:
%%read_sql
SELECT tconst FROM TitleRatings WHERE numVotes = @largestNumVotes;

The following query obtain `tconst` of the largestNumVotes.

In [ ]:
%%read_sql
SET @mostPopularMovie = (
    SELECT tconst FROM TitleRatings WHERE numVotes = @largestNumVotes
);

And the information regarding the title is found as:

In [ ]:
%%read_sql
SELECT * FROM TitleBasics WHERE tconst = @mostPopularMovie;

Note that, variable can store only one column of a record. The following query does not work:

In [ ]:
# This is result in ERROR
%%read_sql
SET @allMovies = (
    SELECT tconst FROM TitleBasics WHERE titleType = "movie"
);

In [ ]:
# This query works (pick tconst of one movie).
%%read_sql
SET @allMovies = (
    SELECT tconst FROM TitleBasics WHERE titleType = "movie" LIMIT 1
);

#### Exercise (Variables)

* Q1. Put the tconst of **primaryTitle** "The Lord of the Rings: The Fellowship of the Ring" (**startYear** = 2001) into a variable `@lotr`.
* Q2. Put the averageRating of "The Lord of the Rings: The Fellowship of the Ring" into a variable `@lotr_rating`.
* Q3. Print `@lotr_rating`.

In [ ]:
# Your answer to Q1 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

In [ ]:
# Your answer to Q2 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

In [ ]:
# Your answer to Q3 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

#### IN clause

The following query returns all the `tconst` where "Brad Pitt" is involved.

In [ ]:
%%read_sql
SELECT tconst FROM TitlePrincipals WHERE nconst =
(
    SELECT nconst FROM NameBasics WHERE primaryName = "Brad Pitt"
) 
;

Is Brad Pitt acting in "Se7en" (1995)? 

In [30]:
%%read_sql
SELECT * FROM TitleBasics WHERE 
primaryTitle = "se7en"
AND 
tconst IN
(
  SELECT tconst FROM TitlePrincipals WHERE nconst =
  (
      SELECT nconst FROM NameBasics WHERE primaryName = "Brad Pitt"
  ) 
)
;

Query started at 10:41:43 AM EDT; Query executed in 0.01 m

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,111777,tt0114369,movie,Se7en,Se7en,0,1995,None,127,"Crime,Drama,Mystery"


Is Brad Pitt acting in "The Hobbit: An Unexpected Journey" (2012)? 


In [31]:
%%read_sql
SELECT * FROM TitleBasics WHERE 
primaryTitle = "The Hobbit: An Unexpected Journey"
AND 
tconst IN
(
  SELECT tconst FROM TitlePrincipals WHERE nconst =
  (
      SELECT nconst FROM NameBasics WHERE primaryName = "Brad Pitt"
  ) 
)
;

Query started at 10:41:47 AM EDT; Query executed in 0.00 m

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


#### Case (conditional construct)

Suppose I am interested in the producers. Create **an column "is_actress"** that describes whether the person is an actress or not.

In [32]:
%%read_sql
SELECT primaryName, 
  CASE 
  WHEN primaryProfession LIKE "%%actress%%" THEN "Yes"
  ELSE "No"
  END AS "is_actress"
FROM NameBasics
where primaryName in ("Hugh Jackman", "Ian McKellen", "Famke Janssen", "Halle Berry")
;

Query started at 10:41:51 AM EDT; Query executed in 0.00 m

,primaryName,is_actress
0,Famke Janssen,Yes
1,Halle Berry,Yes
2,Halle Berry,No
3,Hugh Jackman,No
4,Hugh Jackman,No
5,Ian McKellen,No


#### Functions



The following is the **rounded** values of average ratings of "X-Men" movies.

In [33]:
%%read_sql
SELECT averageRating, ROUND(averageRating, 0) FROM TitleRatings 
WHERE tconst IN
( 
    SELECT tconst FROM TitleBasics WHERE
    primaryTitle IN 
    (
    "X-Men", "X2: X-Men United", "X-Men: The Last Stand", 
    "X-Men Origins: Wolverine", "X-Men: First Class",
    "The Wolverine", "X-Men: Days of Future Past",
    "X-Men: Apocalypse"," X-Men: Dark Phoenix"
    )
    AND
    titleType = "movie"
)
;

Query started at 10:41:54 AM EDT; Query executed in 0.01 m

,averageRating,"ROUND(averageRating, 0)"
0,7.4,7.0
1,6.7,7.0
2,7.3,7.0
3,6.5,6.0
4,6.9,7.0
5,7.9,8.0
6,7.7,8.0
7,6.6,7.0
8,7.4,7.0


#### Exercise

Q1. In **TitleRatings** table, create an column "high_rated" which is "Yes" if averageRating > 7.0 and "No" otherwise.

Q2. In **TitleBasics** table, create an column "high_rated" which is "Yes" if averageRating > 7.0 and "No" otherwise (Hint: JOIN or subquery with TitleRatings table). Check the "X-Men" movies with the following condition.


```
WHERE 
primaryTitle IN 
(
 "X-Men", "X2: X-Men United", "X-Men: The Last Stand", 
 "X-Men Origins: Wolverine", "X-Men: First Class",
 "The Wolverine", "X-Men: Days of Future Past",
 "X-Men: Apocalypse","X-Men: Dark Phoenix",
 "The New Mutants"
)
AND
titleType = "movie"
```



In [ ]:
# Your answer to Q1 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

In [ ]:
# Your answer to Q2 here
%%read_sql
# REPLACE THIS COMMENT WITH SQL

The following is under construction...

## Common Table Expressions

Q. Find all movies where Akira Kurosawa participated. Here, use CTE.

In [34]:
%%read_sql
WITH AK_tconsts AS
(
    SELECT tconst FROM TitlePrincipals WHERE nconst IN
    (
        SELECT nconst FROM NameBasics WHERE
          primaryName = "Akira Kurosawa"
    )
)
SELECT primaryTitle, originalTitle FROM TitleBasics
WHERE tconst IN (SELECT * FROM AK_tconsts)
AND 
  titleType = "movie"
;

Query started at 10:42:02 AM EDT; Query executed in 0.01 m

,primaryTitle,originalTitle
0,Uma,Uma
1,Sanshiro Sugata,Sugata Sanshirô
2,The Most Beautiful,Ichiban utsukushiku
3,The Men Who Tread on the Tiger's Tail,Tora no o wo fumu otokotachi
4,"Sanshiro Sugata, Part Two",Zoku Sugata Sanshirô
...,...,...
72,The Outrage,U mong pa meung
73,Soman kokkyô 2-gô sakusen: Kieta chûtai,Soman kokkyô 2-gô sakusen: Kieta chûtai
74,The Magnificent Seven,The Magnificent Seven
75,Living,Living


When we use CTE? 


*   You do not need to use CTE for single-line query (subquery is enough), but it better organizes a very long query.
*   CTE can be reused within the statement once, twice or more times.

